# `System.DateTime` and JavaScript dates

Here I will write down some basic observations about the `DateTime` `struct` in .NET [📖 [docs](https://learn.microsoft.com/en-us/dotnet/api/system.datetime?view=net-7.0)] and the `Date` object in JavaScript [📖 [docs](https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Global_Objects/Date)]. These remarks appear here after programming for 30 years so I apologize in advance for not having any depth in this area as much as, say, Jon Skeet.

## .NET: there is no need to call `DateTime.Now.ToUniversalTime()`

There is no need to call `DateTime.Now.ToUniversalTime()` because we have `DateTime.UtcNow`:

In [1]:
var utcNow = DateTime.Now.ToUniversalTime().ToString();

utcNow

10/22/2023 5:01:05 AM

In [2]:
DateTime.UtcNow.ToString()

10/22/2023 5:01:05 AM

## JavaScript: `Date.parse` does _not_ return a `Date` object

In [5]:
#!javascript

#!share --from csharp utcNow
const timestamp = Date.parse(utcNow);

console.log(timestamp instanceof Date); // not a Date
console.log(typeof timestamp);          // just a number
console.log(timestamp);                 // just a number, really
console.log(new Date(timestamp));       // returns a locale date and time

false

number

1697976065000

The last output of the JavaScript above is a locale based date. How do we get back to the UTC date shared from the .NET code above? Answering this question reminds me of why JavaScript has a few “good parts.”

## JavaScript: the `Date.UTC()` static method does not take timestamp numbers

The [MDN docs](https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Global_Objects/Date/UTC#behavior_of_date.utc_with_one_argument) get to the point:

>`Date.UTC()` when passed one argument used to have inconsistent behavior, because implementations only kept the behavior consistent with the `Date()` constructor, which does not interpret a single argument as the year number.

Being “consistent with the `Date()` constructor” means changing the meaning of the first argument from `year` to `timestamp`. This anomalous detail is now gone from JavaScript.

## JavaScript: the `Date` constructor takes “a simplification of the ISO 8601 calendar date extended format”

According to [the MDN docs](https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Global_Objects/Date#date_time_string_format), this is the format template that is the “one format to be universally supported”:

```plaintext
YYYY-MM-DDTHH:mm:ss.sssZ
```

We have seen this format in the JavaScript output above. This implies that the date string format we have shared from .NET, in the `utcNow` variable, is _not_ the correct format but is still somehow supported by JavaScript:

In [7]:
#!javascript

#!share --from csharp utcNow

console.log(utcNow);
console.log(new Date(utcNow));

10/22/2023 5:01:05 AM

## .NET: using the “universally supported” JavaScript format

Does the “universally supported” template, `YYYY-MM-DDTHH:mm:ss.sssZ`, work in .NET verbatim?

In [9]:
DateTime.Parse(utcNow).ToString("YYYY-MM-DDTHH:mm:ss.sssZ")

YYYY-10-DDT05:01:05.05Z

The short answer is _no_. After a review of “[Custom date and time format strings](https://learn.microsoft.com/en-us/dotnet/standard/base-types/custom-date-and-time-format-strings),” we may see this:

```plaintext
yyyy'-'MM'-'dd'T'HH':'mm':'ss'.'fff'Z'
```

In [10]:
DateTime.Parse(utcNow).ToString("yyyy'-'MM'-'dd'T'HH':'mm':'ss'.'fff'Z'")

2023-10-22T05:01:05.000Z

It is important to mention that this .NET template for the [ISO 8601](https://en.wikipedia.org/wiki/ISO_8601) format is _not_ among the “[Standard date and time format strings](https://learn.microsoft.com/en-us/dotnet/standard/base-types/standard-date-and-time-format-strings)” of .NET. And this template can easily be confused with `UniversalSortableDateTimePattern` [📖 [docs](https://learn.microsoft.com/en-us/dotnet/api/system.globalization.datetimeformatinfo.universalsortabledatetimepattern?view=net-7.0)], `yyyy'-'MM'-'dd HH':'mm':'ss'Z'`.

## JavaScript: `getUTCDate()` _does_ have something to do with converting an instance of `Date` to UTC


`getUTCDate()` [📖 [docs](https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Global_Objects/Date/getUTCDate)] does have something to do with converting an instance of `Date` to UTC—but _not_ in the way one would assume. According to a very popular StackOverflow answer, the way `getUTCDate()` is used is “[simple and stupid](https://stackoverflow.com/a/6777470/22944)”:

In [25]:
#!javascript

#!share --from csharp utcNow

const convertToUtc = date => {
    return Date.UTC
        (
            date.getUTCFullYear(),
            date.getUTCMonth(),
            date.getUTCDate(),
            date.getUTCHours(),
            date.getUTCMinutes(),
            date.getUTCSeconds(),
            date.getUTCMilliseconds()
        );
};

const utcDate = convertToUtc(new Date());

console.log(utcDate);
console.log(utcDate.toUTCString());
console.log(utcDate.toISOString());

Sun, 22 Oct 2023 19:01:05 GMT

2023-10-22T19:01:05.000Z

We have finally returned the very first UTC date string we got from .NET at the beginning of this notebook!

## <!-- -->

🐙🐱[BryanWilhite](https://github.com/BryanWilhite)